# Fraud Detection
This project will fit and assess a model to predict whether a transaction is a fraud or not.

**Data sources**

- Kaggle: [Synthetic Financial Datasets For Fraud Detection](https://www.kaggle.com/datasets/ealaxi/paysim1) 

For this project, instead of using the file `transactions.csv` that contains 200,000 observations, a modified dataset (`transactions_modified.csv`) will be used which contains only 1,000 observations.

## Import Python Modules

First, import the preliminary modules that will be used in this project:

In [2]:
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler